In [81]:
import os
import pandas as pd
import config_data
import datetime
from unidecode import unidecode
from urllib.parse import unquote

In [2]:
countryto_countrynospace = {key.lower():key.replace(" ","").lower() for key in config_data.country_to_city.keys()}
countrylowerto_citylower = {key.lower():value.lower() for key,value in config_data.country_to_city.items()}
cityto_citynospace = {value.lower():value.replace(" ","").replace(".","").lower() for key,value in config_data.country_to_city.items()}
countrynospaceto_citynospace = {key.replace(" ",""):value.replace(" ","").replace(".","") for key, value in countrylowerto_citylower.items()}
countrynospaceto_citynospace

us_stateto_citynospace = {}

In [93]:
def proc_dirto_row(loc_dir):
    #read all CSVs in loc_dir and retrieve the number of unique listings in each CSV
    timewise_dict = {}
    min_year =2020
    min_month = 12
    max_year = 0
    max_month = 1
    for csvfile in os.listdir(loc_dir):
        csvfile_fullpath = os.path.join(loc_dir,csvfile)
        if csvfile.split('.')[-1] == "csv":
            try:
                df = pd.read_csv(csvfile_fullpath)
                csvfile_no_ext = csvfile[:csvfile.rfind('.')]
                year, month, day = tuple(csvfile_no_ext.split('-')[-3:])
                year = int(year)
                month = int(month)
                if year <= min_year:
                    min_year = year
                    if month < min_month:
                        min_month = month
                if year >= max_year:
                    max_year = year
                    if month > max_month:
                        max_month = month
                k = (year, month)
                timewise_dict[k]= len(pd.unique(df['id']))
#                 print(f"File Date:{(month,year)} Number of unique listings:{len(pd.unique(df['id']))}")
            except OSError:
                print(f"Skipping {csvfile_fullpath}")
    sorted_by_time = [(k,timewise_dict[k]) for k in sorted(timewise_dict.keys())]
    sorted_by_time = sorted_by_time[::-1]
    print(sorted_by_time)

In [92]:
#this snippet can also be used while scraping data to only retrieve data that is going to be used
#other countries and their cities that are not going to be used can be skipped
#at download time itself!

intl_row_dict = dict()
us_row_dict = dict()
us_state_row_dict = dict()

for loc_dir in os.listdir(config_data.AIRBNB_LISTINGS_DATA_DIR):
    loc_dir_fullpath = os.path.join(config_data.AIRBNB_LISTINGS_DATA_DIR, loc_dir)
    print(loc_dir_fullpath)
    mergedname = unidecode(unquote(loc_dir.replace("-", "")))
    for k,v in countryto_countrynospace.items():
        if v in mergedname:
            withoutcountryname = mergedname[mergedname.find(v) + len(v):]
            state = ''
            if v == 'unitedstates':
                state_abbrev = withoutcountryname[:2]
                state_name = config_data.abbrevto_us_state[state_abbrev]
                if state_name in config_data.US_state_to_city.keys():
                    #handle the among state level dataframe here
                    #check if it is a city in the list
                    for city in config_data.US_state_to_city[state_name]:
                        citylower_nospace = city.lower().replace(" ","")
                        if citylower_nospace in mergedname:
                            print(f"getting row for {city} for within US")
                            cityrow = proc_dirto_row(loc_dir_fullpath)
                            us_row_dict[state_name] = cityrow
                            if state_name.lower() == 'california':
                                #also append the row to the within-state dict
                                print(f"getting row for {city} for within Cali")
                                us_state_row_dict[city] = cityrow
                                
            #for all countries, check if it is the representative city:
            if countrynospaceto_citynospace[v] in mergedname:
                print(f"getting row for {v} for international")
                #process the loc_dir to get the dataframe
                intl_row_dict[v] = proc_dirto_row(loc_dir_fullpath)

./AirbnbData/Airbnb_Listings_Gz/spain-catalonia-barcelona
./AirbnbData/Airbnb_Listings_Gz/canada-on-ottawa
getting row for canada for international
[((2020, 10), 2613), ((2020, 9), 2631), ((2020, 8), 2681), ((2020, 6), 2899), ((2020, 5), 2949), ((2020, 4), 3015), ((2020, 3), 3237), ((2020, 2), 3291), ((2020, 1), 3293), ((2019, 12), 3367), ((2019, 11), 3310), ((2019, 10), 3236), ((2019, 9), 3206), ((2019, 7), 3268), ((2019, 6), 3173), ((2019, 5), 3087), ((2019, 3), 2996), ((2019, 2), 2830)]
./AirbnbData/Airbnb_Listings_Gz/united-states-ma-cambridge
getting row for Cambridge for within US
[((2020, 10), 694), ((2020, 6), 1029), ((2020, 5), 1076), ((2020, 4), 1117), ((2020, 3), 1180), ((2020, 2), 1188), ((2020, 1), 1196), ((2019, 12), 1191), ((2019, 11), 1277), ((2019, 10), 1260), ((2019, 9), 1304), ((2019, 8), 1327), ((2019, 7), 1377), ((2019, 6), 1369), ((2019, 5), 1372), ((2019, 4), 1350), ((2019, 3), 1338), ((2019, 2), 1304), ((2019, 1), 1272), ((2018, 12), 1325), ((2018, 11), 1300)]
.

[((2020, 10), 10071), ((2020, 6), 10858), ((2020, 5), 10733), ((2020, 4), 10801), ((2020, 3), 11078), ((2020, 2), 11192), ((2020, 1), 10685), ((2019, 12), 10566), ((2019, 11), 10325), ((2019, 10), 10330), ((2019, 9), 10229), ((2019, 8), 10010), ((2019, 7), 9577), ((2019, 6), 9240), ((2019, 5), 9165), ((2019, 4), 8695), ((2019, 3), 8495)]
./AirbnbData/Airbnb_Listings_Gz/ireland-2020-03-20
./AirbnbData/Airbnb_Listings_Gz/italy-emilia-romagna-bologna
./AirbnbData/Airbnb_Listings_Gz/czech-republic-prague-prague
./AirbnbData/Airbnb_Listings_Gz/united-kingdom-england-london
./AirbnbData/Airbnb_Listings_Gz/the-netherlands-north-holland-amsterdam
./AirbnbData/Airbnb_Listings_Gz/switzerland-geneva-geneva
./AirbnbData/Airbnb_Listings_Gz/belize-bz-belize
./AirbnbData/Airbnb_Listings_Gz/ireland-2020-08-26
./AirbnbData/Airbnb_Listings_Gz/france-auvergne-rhone-alpes-lyon
./AirbnbData/Airbnb_Listings_Gz/ireland-2019-05-12
./AirbnbData/Airbnb_Listings_Gz/ireland-2020-05-27
./AirbnbData/Airbnb_Listings

[((2020, 10), 2082), ((2020, 6), 3204), ((2020, 5), 3281), ((2020, 4), 3360), ((2020, 3), 3437), ((2020, 2), 3387), ((2020, 1), 3383), ((2019, 12), 3379), ((2019, 11), 3350), ((2019, 10), 3288), ((2019, 9), 3236), ((2019, 8), 3243), ((2019, 7), 3211), ((2019, 6), 3186), ((2019, 5), 3167), ((2019, 4), 3103), ((2019, 3), 3068), ((2019, 2), 3064), ((2019, 1), 3055), ((2018, 12), 3024), ((2018, 11), 3053), ((2018, 10), 2977), ((2018, 9), 2932), ((2018, 8), 2898), ((2018, 7), 2829), ((2018, 5), 2826), ((2018, 4), 3131), ((2016, 5), 1718), ((2015, 6), 1155)]
getting row for Oakland for within Cali
./AirbnbData/Airbnb_Listings_Gz/united-states-oh-columbus
getting row for Columbus for within US
[((2020, 10), 1157), ((2020, 7), 1409), ((2020, 6), 1518), ((2020, 5), 1558), ((2020, 4), 1620), ((2020, 3), 1683), ((2020, 2), 1586), ((2020, 1), 1462), ((2019, 12), 1487), ((2019, 11), 1411), ((2019, 10), 1307), ((2019, 9), 1248), ((2019, 8), 1254), ((2019, 7), 1210), ((2019, 6), 1260), ((2019, 5), 13

'1.1.4'